In [1]:
from tune import Space, Grid, Rand, RandInt, Choice

In [2]:
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

# Example 1: Union Space

`+` means take the union of the spaces

In [3]:
# use case: different tuning method on different modeling algorithms

xgb_gird = Space(model=XGBRegressor, n_estimatores=Grid(50,150))  
lgbm_random = Space(model=LGBMRegressor, n_estimatores=RandInt(100,200)).sample(3) 
catboost_bo = Space(model=CatBoostRegressor, n_estimatores=RandInt(100,200))  

union_space = xgb_gird + lgbm_random + catboost_bo # "+" takes the union of spaces

list(union_space)

[{'model': <class 'xgboost.sklearn.XGBRegressor'>, 'n_estimatores': 50},
 {'model': <class 'xgboost.sklearn.XGBRegressor'>, 'n_estimatores': 150},
 {'model': <class 'lightgbm.sklearn.LGBMRegressor'>, 'n_estimatores': 100},
 {'model': <class 'lightgbm.sklearn.LGBMRegressor'>, 'n_estimatores': 146},
 {'model': <class 'lightgbm.sklearn.LGBMRegressor'>, 'n_estimatores': 148},
 {'model': <class 'catboost.core.CatBoostRegressor'>, 'n_estimatores': RandInt(low=100, high=200, q=1, log=False, include_high=True)}]

# Example 2: Cross Product Space

`*` means take the cross product of the spaces

In [4]:
# use case: diffferent tuning method inside one modeling algorithm

non_bo_space = Space(
    model=LGBMRegressor, 
    n_estimators=100,
    boosting=Grid("dart", "gbdt"),    # Grid search
    feature_fraction=Rand(0.5, 1)     # Random search
).sample(2, seed=0) 

bo_space = Space(
    learning_rate=Rand(1e-8, 10, log=True)  # Bayesian Optimization
) 

product_space = non_bo_space * bo_space # "+" takes cross product of spaces

list(product_space)

[{'model': <class 'lightgbm.sklearn.LGBMRegressor'>, 'n_estimators': 100, 'boosting': 'dart', 'feature_fraction': 0.7744067519636624, 'learning_rate': Rand(low=1e-08, high=10, q=None, log=True, include_high=True)},
 {'model': <class 'lightgbm.sklearn.LGBMRegressor'>, 'n_estimators': 100, 'boosting': 'gbdt', 'feature_fraction': 0.7744067519636624, 'learning_rate': Rand(low=1e-08, high=10, q=None, log=True, include_high=True)},
 {'model': <class 'lightgbm.sklearn.LGBMRegressor'>, 'n_estimators': 100, 'boosting': 'dart', 'feature_fraction': 0.8575946831862098, 'learning_rate': Rand(low=1e-08, high=10, q=None, log=True, include_high=True)},
 {'model': <class 'lightgbm.sklearn.LGBMRegressor'>, 'n_estimators': 100, 'boosting': 'gbdt', 'feature_fraction': 0.8575946831862098, 'learning_rate': Rand(low=1e-08, high=10, q=None, log=True, include_high=True)}]

# Example 3: Construct Hybrid Search Space

Now we can use these operators to construct any combination of searching spaces as desired.

In [5]:
# use case: any kind of hybrid search spaces
# e.g. tuning a common parameter on 3 modeling algorithms

xgb_static = Space(model=XGBRegressor)
lgb_static = Space(model=LGBMRegressor)
catboost_static = Space(model=CatBoostRegressor)

bo_space = Space(n_estimatores=RandInt(100,200)) # Bayesian Optimization on a common parameter

hybrid_space = (xgb_static + lgb_static + catboost_static) * bo_space

list(hybrid_space)

[{'model': <class 'xgboost.sklearn.XGBRegressor'>, 'n_estimatores': RandInt(low=100, high=200, q=1, log=False, include_high=True)},
 {'model': <class 'lightgbm.sklearn.LGBMRegressor'>, 'n_estimatores': RandInt(low=100, high=200, q=1, log=False, include_high=True)},
 {'model': <class 'catboost.core.CatBoostRegressor'>, 'n_estimatores': RandInt(low=100, high=200, q=1, log=False, include_high=True)}]